In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows = None
pd.options.display.max_columns = None
from sklearn.cross_validation import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#import lightgbm as lgb 
import math as mt
from sklearn.ensemble import GradientBoostingClassifier
import catboost as cb
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Digraph
import pydot
import matplotlib.image as mpimg
from IPython.display import Image, display

from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import webbrowser

In [22]:
header_df = pd.read_csv('header.csv')
train_df = pd.read_csv('train.csv',header=None)
test_df = pd.read_csv('test.csv',header=None)
sample_sub = pd.read_csv('sample_submission.csv')

In [23]:
train_df.columns = header_df.columns 
col_list = []

for item in header_df.columns.values :
    if item != 'label' :
           col_list.append(item) 
            
test_df.columns = col_list
test_df.head()

,key,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54
0,258918,2352,293,23,256,45,201,151,228,215,799,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,258134,2790,88,8,330,35,1400,232,227,128,726,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,165265,2826,115,7,162,24,1020,232,233,134,2044,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,522141,3234,137,16,150,20,968,243,232,112,2536,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,569662,2810,58,22,124,25,3293,230,188,80,1624,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
def preprocess(train_df,test_df) :
    cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10']
    for col in cols :
        train_df.loc[:,col] = (train_df[col]-train_df[col].mean())/train_df[col].std()
        test_df.loc[:,col] = (test_df[col]-test_df[col].mean())/test_df[col].std()
    train_df.drop(['V50','V23'],axis=1,inplace=True)
    test_df.drop(['V50','V23'],axis=1,inplace=True)
    train_df['V6_V1'] = train_df['V6'] - train_df['V1']
    test_df['V6_V1'] = test_df['V6'] - test_df['V1']
    train_df['V10_V1'] = train_df['V10'] - train_df['V1']
    test_df['V10_V1'] = test_df['V10'] - test_df['V1']
    train_df['V6_V10_minus'] = train_df['V6'] - train_df['V10']
    test_df['V6_V10_minus'] = test_df['V6'] - test_df['V10']
    train_df['V4_V1'] = train_df['V4'] - train_df['V1']
    test_df['V4_V1'] = test_df['V4'] - test_df['V1']
    train_df['V5_V1'] = train_df['V5'] - train_df['V1']
    test_df['V5_V1'] = test_df['V5'] - test_df['V1']
    train_df['V5_V4'] = train_df['V5'] - train_df['V4']
    test_df['V5_V4'] = test_df['V5'] - test_df['V4']
    train_df['V6_V4'] = train_df['V6'] - train_df['V4']
    test_df['V6_V4'] = test_df['V6'] - test_df['V4']
    train_df['V6_V5'] = train_df['V6'] - train_df['V5']
    test_df['V6_V5'] = test_df['V6'] - test_df['V5']
    train_df['V9_V8'] = train_df['V9'] - train_df['V8']
    test_df['V9_V8'] = test_df['V9'] - test_df['V8']
    train_df['V2_V10'] = train_df['V2'] - train_df['V10']
    test_df['V2_V10'] = test_df['V2'] - test_df['V10']
    train_df['V1_V11'] = train_df['V1'] * train_df['V11']
    test_df['V1_V11'] = test_df['V1'] * test_df['V11']
    train_df['V6_V13'] = train_df['V6'] * train_df['V13']
    test_df['V6_V13'] = test_df['V6'] * test_df['V13']
    train_df['V9_V47'] = train_df['V9'] * train_df['V47']
    test_df['V9_V47'] = test_df['V9'] * test_df['V47']
    train_df['V10_V3'] = train_df['V10'] * train_df['V13']
    test_df['V10_V3'] = test_df['V10'] * test_df['V13']
    train_df['V6_V10'] = train_df['V6'] / train_df['V10']
    test_df['V6_V10'] = test_df['V6'] / test_df['V10']
    train_df['cat1'] = train_df['V1'] <= -1.63
    test_df['cat1'] = test_df['V1'] <= -1.63
    train_df['cat2'] = (train_df['V1'] > -1.63) & (train_df['V1'] <= 0.457) & (train_df['V1_V11'] <= -0.014)
    test_df['cat2'] = (test_df['V1'] > -1.63) & (test_df['V1'] <= 0.457) & (test_df['V1_V11'] <= -0.014)
    train_df['cat3'] = (train_df['V1'] > -1.63) & (train_df['V1'] <= 0.457) & (train_df['V1_V11'] > -0.014)
    test_df['cat3'] = (test_df['V1'] > -1.63) & (test_df['V1'] <= 0.457) & (test_df['V1_V11'] > -0.014)
    train_df['cat4'] = (train_df['V1'] > 0.457) & (train_df['V1'] <= 1.204)
    test_df['cat4'] = (test_df['V1'] > 0.457) & (test_df['V1'] <= 1.204)
    train_df['cat5'] = train_df['V1'] > 1.204
    test_df['cat5'] = test_df['V1'] > 1.204
    train_df['cat6'] = train_df.V6_V1 <= 0.916
    test_df['cat6'] = test_df.V6_V1 <= 0.916
    train_df['cat7'] = train_df.V6_V1 <= 0.916
    test_df['cat7'] = test_df.V6_V1 <= 0.916
    return train_df,test_df

In [25]:
train_df,test_df = preprocess(train_df,test_df)
train_x, dev_x, train_y, dev_y = train_test_split(train_df.drop(['key','label'],axis=1), train_df['label'],test_size=0.33, random_state=42)

In [20]:
train_df['cat2'].value_counts()

False    335064
True      71645
Name: cat2, dtype: int64

In [39]:
#cv_params = {'max_depth': [10,20,100], 'min_child_weight': [3,5,10],'subsample' : [0.7,0.8,0.9],'learning_rate':[0.05,0.1,0.2]}
#ind_params = { 'n_estimators': 100, 'seed':0,  'objective': 'binary:logistic'}
#optimized_GBM = RandomizedSearchCV(xgb.XGBClassifier(**ind_params), 
#                            cv_params, 
#                             scoring = 'roc_auc', cv = 3, n_jobs = -1,n_iter=10) 

In [40]:
#optimized_GBM.fit(train_df.drop(['key','label'],axis=1), train_df['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'subsample': [0.7, 0.8, 0.9], 'learning_rate': [0.05, 0.1, 0.2], 'max_depth': [10, 20, 100], 'min_child_weight': [3, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [41]:
#optimized_GBM.grid_scores_

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99448, std: 0.00024, params: {'subsample': 0.9, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 3},
 mean: 0.99556, std: 0.00018, params: {'subsample': 0.9, 'learning_rate': 0.1, 'max_depth': 100, 'min_child_weight': 5},
 mean: 0.99335, std: 0.00025, params: {'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 5},
 mean: 0.97206, std: 0.00074, params: {'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 10},
 mean: 0.98808, std: 0.00041, params: {'subsample': 0.8, 'learning_rate': 0.05, 'max_depth': 20, 'min_child_weight': 10},
 mean: 0.98715, std: 0.00044, params: {'subsample': 0.7, 'learning_rate': 0.05, 'max_depth': 20, 'min_child_weight': 10},
 mean: 0.99417, std: 0.00018, params: {'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 3},
 mean: 0.99083, std: 0.00018, params: {'subsample': 0.7, 'learning_rate': 0.05, 'max_depth': 20, 'min_child_weight': 5},
 mean: 0.99562, std: 0.00015, par

In [42]:
#with open('model1.param','w') as fh :
#    fh.write(str(optimized_GBM.grid_scores_))

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [26]:
#mean: 0.99562, std: 0.00015, params: {'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 100, 'min_child_weight': 3},
clf = XGBClassifier(max_depth=100,subsample=0.8,eta=0.1,seed=0,n_estimators=500,min_child_weight=3)
clf.fit(train_df.drop(['key','label'],axis=1),train_df['label'])
#pred = clf.predict(dev_x)
#print roc_auc_score(dev_y,pred)
print roc_auc_score(train_df['label'],clf.predict(train_df.drop(['key','label'],axis=1)))

1.0


/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [27]:
key_test = test_df['key']
test_df = test_df.drop('key',axis=1)
pred_test = clf.predict_proba(test_df)

In [28]:
final_pred = []
for i in range(0,len(pred_test)) :
    final_pred.append(pred_test[i][1])

In [29]:
test_xgb = pd.DataFrame({
        "key": key_test,
        "score": final_pred
    })

In [30]:
test_xgb.to_csv('final_model.csv',index=False)